In [1]:
# Enable automatic reloading of modules before executing code
%load_ext autoreload
%autoreload 2

In [176]:
import os
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [3]:
import utils as ut

In [19]:
# test run of the model for dev purposes, or full/final data set?
is_testcase = True

# Load Model Results from Disk

In [4]:
l_lmbda = [0.0005]
l_lmbda_hierarchical = [0.01]
l_rnd_seed = [7640]
l_embed_dim = [7]
l_sparse = ["items_and_random_ids"]
modeltype = "random_weights_random_scaling"
modelversion = "embeddings-decision-combined-data"

In [25]:
# load results from model run
l_results = load_ID_lowdim(
    l_lmbda=l_lmbda, l_lmbda_hierarchical=l_lmbda_hierarchical, l_rnd_seed=l_rnd_seed, 
    l_embed_dim=l_embed_dim, l_sparse=l_sparse, modelversion=modelversion, modeltype=modeltype
)

In [39]:
# Load Relevant IDs for correlational analyses
if is_testcase:
    df_pids = pd.read_csv("data/study1-2025-08/new-participant-ids-in-joint-modeling-testcase.csv")
elif not is_testcase:
    df_pids = pd.read_csv("data/study1-2025-08/new-participant-ids-in-joint-modeling.csv")

In [52]:
l_pids_model = list(df_pids["participant_id_model"])
l_pids_new = list(df_pids["participant_id_new"])

In [277]:
df_dim_weights = pd.DataFrame(l_results[0]["decision_weights"].detach().numpy()[l_pids_model, :])
# participant_id_new can be joined with results coming only from the new study
df_dim_weights["participant_id_new"] = l_pids_new
# Move participant_id column to the first position
col = "participant_id_new"
new_order = [col] + [c for c in df_dim_weights.columns if c != col]
df_dim_weights = df_dim_weights[new_order]
df_dim_weights["avg_weight"] = df_dim_weights[0:6].mean(axis=0)

In [248]:
# load dfs saved with R
df_qs_num_long = pd.read_csv("data/study1-2025-08/tbl_qs_num_long_excluded.csv")
df_qs_txt = pd.read_csv("data/study1-2025-08/tbl_qs_txt_excluded.csv")
df_qs_num = df_qs_num_long.pivot(index=["participant_id", "session_id"], columns="name", values="value").reset_index(drop=False)
# reverse code items
df_qs_num = ut.reverse_code_q_items(df_qs_num)
# create scale and facet scores
df_qs_num = ut.scales_and_facets(df_qs_num)
# only keep scales, drop items and facets
df_qs_num = ut.keep_scales_only(df_qs_num)

In [249]:
model_id = "answerdotai/ModernBERT-base"
model, tokenizer, device = ut.load_model(model_id)
df_qs_txt = ut.add_prefixes_and_concatenate_cols(df_qs_txt)

df_qs_txt["embedding_work_history"] = df_qs_txt["workHistory"].apply(
    lambda col: ut.tokenize_col(
        col, prefix1 = "Here are some details about my life: ", 
        prefix2 = "I have worked as ", 
        tokenizer=tokenizer, model=model, device=device
    )
)
df_qs_txt["embedding_interests1"] = df_qs_txt["interests1"].apply(
    lambda col: ut.tokenize_col(
        col, prefix1 = "Here are some details about my life: ", 
        prefix2 = "The topics or activities I am most passionate about are: ", 
        tokenizer=tokenizer, model=model, device=device
    )
)
df_qs_txt["embedding_interests2"] = df_qs_txt["interests2"].apply(
    lambda col: ut.tokenize_col(
        col, prefix1 = "Here are some details about my life: ", 
        prefix2 = "The topics or activities I love doing in my free time are: ", 
        tokenizer=tokenizer, model=model, device=device
    )
)
df_qs_txt["embedding_interests3"] = df_qs_txt["interests3"].apply(
    lambda col: ut.tokenize_col(
        col, prefix1 = "Here are some details about my life: ", 
        prefix2 = "The subjects, which fascinate me and make me want to learn more are: ", 
        tokenizer=tokenizer, model=model, device=device
    )
)
df_qs_txt["embedding_all_txt"] = df_qs_txt["txt_concat"].apply(
    lambda col: ut.tokenize_col(col, prefix1 = "", prefix2 = "", tokenizer=tokenizer, model=model, device=device)
)

In [271]:
idx_embeddings = [col.startswith("embedding") for col in df_qs_txt.columns]
colnames_required = list(df_qs_txt.columns[idx_embeddings])

colnames_required.extend(["participant_id", "session_id"])
df_embeddings = df_qs_txt[colnames_required]
df_qs_num = df_qs_num.merge(df_embeddings, how="left", on=["participant_id", "session_id"])

In [279]:
df_ooo_qs = df_dim_weights.merge(df_qs_num, how="left", left_on="participant_id_new", right_on="participant_id")
df_ooo_qs.rename(columns={"participant_id_new":"pid"}, inplace=True)

*todos*

- run model with split-half data
- create features with dim weights:
    - gini coef
    - load results and create feature "stability" of weights between halfs